In [1]:
import numpy as np
import sys

In [2]:
# initiate a random population of size N. Always starting and ending the journey at 0
def rand_init(N, nbr_cities):
    pop = np.zeros((1, nbr_cities + 1), dtype=int)
    pop[0, 1:nbr_cities] = np.random.permutation(nbr_cities - 1) + 1
    for _ in range(N - 1):
        new_p = np.zeros((1, nbr_cities + 1), dtype=int)
        new_p[0, 1:nbr_cities] = np.random.permutation(nbr_cities - 1) + 1
        pop = np.concatenate((pop, new_p))

    # print('rand_init gives: ', pop)
    return pop

In [3]:
# evaluate performance of phenotypes in pop
def evaluate(pop, dist_mat):
    N = len(pop)
    dists = np.zeros(N)
    for n, geno in enumerate(pop):
        for i in range(len(geno) - 1):
            dists[n] += dist_mat[geno[i], geno[i+1]]
        
    # print('evaluate gives: ', rank)
    return dists

In [4]:
# Gives a new popolation, two individuals meet and the best is kept.
# so half of the population is killed of at each round and replaced with random individuals.
# if permutate = 1 we shuffle the popolation after the round so the positions are random
# if permute = 0 we will push winning individuals upwards and stack the bottom with the new ones
# A individual who wins its dual will be mutated by a probability of mutation_rate
def update_pop(old_pop, dists, mutation_rate, permutate=False, bootstrap=False):
    N = len(old_pop)
    new_pop = np.zeros(np.shape(old_pop), dtype=int)
    for i in range(int(N / 2)):
        if bootstrap:
            indices = np.random.randint(N, size=2)
        else:
            indices = [2 * i, 2 * i + 1]

        if dists[indices[0]] < dists[indices[1]]:
            survivor = old_pop[indices[0]]
        else:
            survivor = old_pop[indices[1]]
        
        if np.random.uniform() < mutation_rate:
            survivor = mutate(survivor)
            
        new_pop[i] = survivor
    
    # fyll andra halvan med nya samples
    new_pop[int(N/2):, :] = rand_init(int(N/2), nbr_cities)
    
    if permutate and not bootstrap: 
        new_pop = np.random.permutation(new_pop)

    # print('update_pop gives: ', new_pop)
    return new_pop

In [24]:
# How could I do a crossover between two paths? keep the best connections for each?
# def crossover


        


In [5]:
# Works like the other one but uses bootstrap to pick candidates for duals. 
# Meaning that some will fight twice and som will have no chanse of survival.
# It will not matter if I permutate or not but.
def update_pop_bootstrap(old_pop, dists, mutation_rate):
    N = len(old_pop)
    new_pop = np.zeros(np.shape(old_pop), dtype=int)
    for i in range(int(N / 2)):
        indices = np.random.randint(N, size=10)
        
        if dists[indices[0]] < dists[indices[1]]:
            survivor = old_pop[indices[0]]
        else:
            survivor = old_pop[indices[1]]
        
        if np.random.uniform() < mutation_rate:
            survivor = mutate(survivor)
            
        new_pop[i] = survivor
    
    # fyll andra halvan med nya samples
    new_pop[int(N/2):, :] = rand_init(int(N/2), nbr_cities)

            
    # print('update_pop gives: ', new_pop)
    return new_pop

In [6]:
# does the mutation by swiching place of two indices
# Could I introduce a more profound mutation method?
def mutate(geno):
    swaps = np.random.randint(len(geno) - 2, size=2) + 1 
    temp = geno[swaps[0]]
    geno[swaps[0]] = geno[swaps[1]]
    geno[swaps[1]] = temp
    
    return geno

In [7]:
# Returns the best individual of the poputaion
def get_best(pop, dists):
    bi = -1
    min_dist = dists[0] + dists[1]   # ensures that we will find something smaller
    for idx, dist in enumerate(dists):
        if dist < min_dist:
            min_dist = dist
            bi = idx
            
    return pop[bi], min_dist

In [10]:
# runs the algorithm
def GA_solver(Generations, Populations_size, mutation_rate, dist_mat, Perm, Boot):
    nbr_cities = len(dist_mat)

    if Populations_size % 2:
        print('My program does not work for non-even population sizes. LOL')
        sys.exit()

    population = rand_init(Populations_size, nbr_cities)
    print('pop,:\n', population)
    dists = evaluate(population, dist_mat)
    print('dists, :', dists)

    for k in range(Generations):
        if np.min(dists) < 0: # 98
            print('it took ', k, ' generations to find the optimal solution')
            break

        population = update_pop(population, dists, mutation_rate, permutate=Perm, bootstrap=Boot)
        dists = evaluate(population, dist_mat)

        if k % 100 == 0 and k > 0: mutation_rate /= 2


        if k % 50 == 0 and k > 0:
            candidate = get_best(population, dists)
            print('current best candidate has the distance: ', candidate[1])


    print('pop,:\n', population)
    print('dists, :', dists)

    winner = get_best(population, dists)
    print('\nThe best path was: ', winner[0], ' Which had the distance: ', winner[1])


In [11]:
# Runs the GA. ATM this seams like a descent algorithm.

# permutate=False, bootstrap=False, why?? might differ if we change mutation

# Possible improvements: 
# 1: indtroduce a way to execute crossover (but is this an improvement??)
# 2: Modify the way we keep individuals. (Now we do a tornament selection where half is killed off)

# I do like this since I want to compare solution for different solvers 
# and then I need to have the same problem
nbr_cities = 50
# Could be any symetric matrix
# dist_mat = np.array([[0, 20, 42, 35], [20, 0, 30, 34], [42, 30, 0, 12], [35, 34, 12, 0]])
dist_mat = ((3 * np.random.random_sample((nbr_cities, nbr_cities))) ** 8) / 1000       
# using ** etc. to increase variance
dist_mat = np.round(np.matmul(dist_mat.transpose(), dist_mat))


Generations = 1000
Populations_size = 50
mutation_rate = 1/2
permutate = False
bootstrap = False

GA_solver(Generations, Populations_size, mutation_rate, dist_mat, permutate, bootstrap)

pop,:
 [[ 0 28  7 ... 42 13  0]
 [ 0 41 18 ...  8 19  0]
 [ 0 26 20 ... 22 31  0]
 ...
 [ 0 19  6 ... 47 48  0]
 [ 0 17 26 ... 32 21  0]
 [ 0 22 13 ... 38 35  0]]
dists, : [1195. 1484. 1242. 1283. 1301. 1340. 1347. 1212. 1460. 1279. 1331. 1241.
 1241. 1213. 1421. 1251. 1376. 1257. 1373. 1244. 1189. 1195. 1268. 1355.
 1328. 1346. 1247. 1313. 1227. 1359. 1454. 1321. 1423. 1288. 1406. 1348.
 1311. 1172. 1291. 1337. 1382. 1238. 1435. 1279. 1233. 1229. 1309. 1236.
 1304. 1249.]
current best candidate has the distance:  1081.0
current best candidate has the distance:  1023.0
current best candidate has the distance:  1050.0
current best candidate has the distance:  1041.0
current best candidate has the distance:  1004.0
current best candidate has the distance:  1023.0
current best candidate has the distance:  1040.0
current best candidate has the distance:  1057.0
current best candidate has the distance:  1053.0
current best candidate has the distance:  1016.0
current best candidate has the d

In [190]:
v = np.zeros((1,5))
v[:,1:4] = np.random.permutation(3) + 1
u = np.zeros((1,5))
u[:,1:4] = np.random.permutation(3) + 1
np.concatenate((u,v))


array([[0., 3., 2., 1., 0.],
       [0., 2., 3., 1., 0.]])

In [9]:
A = ((3 * np.random.random_sample((20,20))) ** 8) / 1000
A = np.round(np.matmul(A.transpose(), A))

m = np.array([[1,1,1], [2,2,2], [3,3,3]])
np.random.permutation(m)

array([[3, 3, 3],
       [2, 2, 2],
       [1, 1, 1]])

In [20]:
[0:5]

SyntaxError: invalid syntax (<ipython-input-20-d2eb08182742>, line 1)